### Import required libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

### Load the dataset

In [2]:
df = pd.read_excel("../datasets/Predictive Variables Experiments_vMaster (Cleaned).xlsx", sheet_name='Data Set', header=2, engine='openpyxl' )

## Data Preprocessing

#### Numerical Variables

In [3]:
numerical_cols = ["DEAL_SIZE", "PRE_MONEY_VALUATION", "POST_VALUATION", "RAISED_TO_DATE", "Market Size Estimate"]

numerical_vars_with_na = [var for var in numerical_cols if df[var].isnull().sum()>0]


df_cleaned_num = df.dropna(subset=numerical_vars_with_na)
df_cleaned_num.shape, df.shape

((2317, 56), (2380, 56))

#### Temporal Variables

In [4]:
# Temporal variables in train_df_cleaned
temporal_vars = ['DEAL_DATE', 'YEAR_FOUNDED']

# Processing DEAL_DATE, since some entries are Excel's serial date numbers!!
def convert_date(datapoint):

    if isinstance(datapoint, str):
        return pd.to_datetime(datapoint, errors='coerce')
    
    elif isinstance(datapoint, (int, float)):
        return pd.Timestamp('1899-12-30') + pd.to_timedelta(datapoint, unit='D')
    
    return datapoint


df_cleaned_num['DEAL_DATE'] = df_cleaned_num['DEAL_DATE'].apply(convert_date)

df_cleaned_num['YEAR_FOUNDED'].isnull().sum()

17

In [5]:
df_cleaned_num[['TRAIN_OR_TEST', 'COMPANIES', 'COMPANY_WEBSITE', 'COMPANY_CITY']][df_cleaned_num['YEAR_FOUNDED'].isnull()]

,TRAIN_OR_TEST,COMPANIES,COMPANY_WEBSITE,COMPANY_CITY
242,Train,Securly,www.securly.com,Charlotte
276,Train,Reconstruct,www.reconstructinc.com,Menlo Park
395,Train,Imagry,www.imagry.co,San Jose
398,Train,Systum,www.systum.com,Plano
539,Train,Bdeo,www.bdeo.io,Madrid
659,Train,Reconstruct,www.reconstructinc.com,Menlo Park
707,Train,Waybridge,www.waybridge.com,New York
1220,Train,Genies,www.genies.com,West Hollywood
1254,Train,Axios Media,www.axios.com,Arlington
1300,Train,LeaseLock,www.leaselock.com,Marina del Rey


- Duplicate entries are present in the data

In [6]:
df_cleaned_num.drop_duplicates(subset='COMPANIES', keep='first', inplace=True)

In [7]:
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Arcus'] = '2018'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Reconstruct'] = '2016'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Imagry'] = '2015'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Bdeo'] = '2017'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Cover (Financial Software)'] = '2016'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='LeaseLock'] = '2016'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Systum'] = '2015'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Axios Media'] = '2017'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Securly'] = '2012'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Genies'] = '2017'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Waybridge'] = '2019'
df_cleaned_num['YEAR_FOUNDED'][df_cleaned_num['COMPANIES']=='Versatile'] = '2016'

In [8]:
# Formatting the YEAR_FOUNDED column into datetime type
df_cleaned_num['YEAR_FOUNDED'] = pd.to_datetime(df_cleaned_num['YEAR_FOUNDED'].astype(int).astype(str), format='%Y')
df_cleaned_num['FOUNDED_YEAR'] = df_cleaned_num['YEAR_FOUNDED'].dt.year

In [9]:
# Extracting DAYS_FROM_DEAL from 'DEAL_DATE'
df_cleaned_num['DAYS_FROM_DEAL'] = (pd.Timestamp.today() - df_cleaned_num['DEAL_DATE']).dt.days

# Extracting DEAL_MONTH from 'DEAL_DATE'
df_cleaned_num['DEAL_MONTH'] = df_cleaned_num['DEAL_DATE'].dt.month

#### Categorical Variables

In [10]:
# Mapping Valuations
valuation_mappings = {'1. $5M - $25M': 1, '2. $25M - $50M': 2, '3. $50M - $100M': 3,
                      '4. $100M - $150M':4, '5. $400M - $1B':5, '6. >$1B':6}

df_cleaned_num['Valuation Tier'] = df_cleaned_num['Valuation Tier'].map(valuation_mappings)

#### Excluding variables which contain less than 5 entries of a category

In [31]:
# Final Variables

categorical_vars = ["TRAIN_OR_TEST", "COMPANIES", "FOUNDED_YEAR", "DEAL_MONTH", "PRIMARY_INDUSTRY_SECTOR",
                    "Market Size Estimate", "S&M Prior Role", "Finance Prior Role", "Valuation Tier", 
                    "Quality Deals Investors", "Firms: Best Investors", "Firm: Worst", "Top Law Firm"
]


numerical_vars = ["DAYS_FROM_DEAL", "DEAL_SIZE", "PRE_MONEY_VALUATION", "POST_VALUATION", "RAISED_TO_DATE"]

target_var = ["IMPLIED_RETURN"]

In [32]:
df_cleaned_num_cat = df_cleaned_num[temporal_vars+numerical_vars+categorical_vars+target_var]

In [33]:
# Removing the single entry category 'Materials and Resources' from 'PRIMARY_INDUSTRY_SECTOR'
df_cleaned_num_cat = df_cleaned_num_cat[df_cleaned_num_cat['PRIMARY_INDUSTRY_SECTOR'] != 'Materials and Resources']

In [34]:
df_cleaned_num_cat.isnull().sum()

DEAL_DATE                  0
YEAR_FOUNDED               0
DAYS_FROM_DEAL             0
DEAL_SIZE                  0
PRE_MONEY_VALUATION        0
POST_VALUATION             0
RAISED_TO_DATE             0
TRAIN_OR_TEST              0
COMPANIES                  0
FOUNDED_YEAR               0
DEAL_MONTH                 0
PRIMARY_INDUSTRY_SECTOR    0
Market Size Estimate       0
S&M Prior Role             0
Finance Prior Role         0
Valuation Tier             0
Quality Deals Investors    0
Firms: Best Investors      0
Firm: Worst                0
Top Law Firm               0
IMPLIED_RETURN             0
dtype: int64

In [35]:
df_cleaned_num_cat.dropna(inplace=True)

In [36]:
df_cleaned_num_cat.drop('YEAR_FOUNDED', axis=1, inplace=True)
df_cleaned_num_cat.columns

Index(['DEAL_DATE', 'DAYS_FROM_DEAL', 'DEAL_SIZE', 'PRE_MONEY_VALUATION',
       'POST_VALUATION', 'RAISED_TO_DATE', 'TRAIN_OR_TEST', 'COMPANIES',
       'FOUNDED_YEAR', 'DEAL_MONTH', 'PRIMARY_INDUSTRY_SECTOR',
       'Market Size Estimate', 'S&M Prior Role', 'Finance Prior Role',
       'Valuation Tier', 'Quality Deals Investors', 'Firms: Best Investors',
       'Firm: Worst', 'Top Law Firm', 'IMPLIED_RETURN'],
      dtype='object')

In [37]:
for var in categorical_vars:
    df_cleaned_num_cat[var] = df_cleaned_num_cat[var].astype('object')


for var in numerical_vars:
    df_cleaned_num_cat[var] = df_cleaned_num_cat[var].astype('str').astype('float')

df_cleaned_num_cat['IMPLIED_RETURN'] = df_cleaned_num_cat['IMPLIED_RETURN'].astype('float')

#### Final dataset used to train and test the model

In [38]:
df_cleaned_num_cat.to_csv('../datasets/data_cleaned_0912.csv', index=False)

In [39]:
# Extracting the train datapoints
df_train = df_cleaned_num_cat[df_cleaned_num_cat['TRAIN_OR_TEST']=='Train']
df_test = df_cleaned_num_cat[df_cleaned_num_cat['TRAIN_OR_TEST']=='Test']

df_train.drop('TRAIN_OR_TEST', axis=1, inplace=True)
df_test.drop('TRAIN_OR_TEST', axis=1, inplace=True)

df_train.shape, df_test.shape

((1625, 19), (473, 19))

#### Splitting the data into train and test sets

In [40]:
X_train, y_train = df_train.drop(['DEAL_DATE', 'IMPLIED_RETURN'], axis=1), df_train['IMPLIED_RETURN']
X_test, y_test = df_test.drop(['DEAL_DATE', 'IMPLIED_RETURN'], axis=1), df_test['IMPLIED_RETURN']

X_train.shape, X_test.shape

((1625, 17), (473, 17))

In [41]:
# Encoding categorical variables
cat_cols_to_encode = ['PRIMARY_INDUSTRY_SECTOR']

label_encoders = {}

for col in cat_cols_to_encode:
    
    le = LabelEncoder()
    le.fit(X_train[col])

    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

    label_encoders[col] = le

## Baseline Model Building

#### Random Forest Regressor

In [42]:
rf = RandomForestRegressor(n_estimators=120, random_state=2023)
rf.fit(X_train.drop('COMPANIES', axis=1), y_train)

RandomForestRegressor(n_estimators=120, random_state=2023)

#### Model Evaluation on Train set

In [43]:
y_train_predictions = rf.predict(X_train.drop('COMPANIES', axis=1))

mse_train = mean_squared_error(y_train, y_train_predictions)
print(f"Mean Squared Error(MSE): {mse_train}\n")

rmse_train = np.sqrt(mse_train)
print(f"Root Mean Squared Error(RMSE): {rmse_train}\n")

mae_train = mean_absolute_error(y_train, y_train_predictions)
print(f"Mean Absolute Error(MAE): {mae_train}\n")

Mean Squared Error(MSE): 32.117378082655584

Root Mean Squared Error(RMSE): 5.667219607766721

Mean Absolute Error(MAE): 1.8335650736186866



#### Model Evaluation on Test set

In [44]:
y_test_predictions = rf.predict(X_test.drop('COMPANIES', axis=1))

mse_test = mean_squared_error(y_test, y_test_predictions)
print(f"Mean Squared Error(MSE): {mse_test}\n")

rmse_test = np.sqrt(mse_test)
print(f"Root Mean Squared Error(RMSE): {rmse_test}\n")

mae_test = mean_absolute_error(y_test, y_test_predictions)
print(f"Mean Absolute Error(MAE): {mae_test}\n")

Mean Squared Error(MSE): 87.16560688505758

Root Mean Squared Error(RMSE): 9.336252293348632

Mean Absolute Error(MAE): 4.166229740336452



## Saving the predictions

In [45]:
# Create a DataFrame with true values, predicted values, and a column from X_test
predicted_results = pd.DataFrame({
    'Company': X_test['COMPANIES'].values,
    'Actual Return': y_test,
    'Predicted Return': y_test_predictions, 
})

In [46]:
# Assuming your DataFrame is named 'predicted_results' and has columns 'actual' and 'predicted'
predicted_results['Actual Rank'] = predicted_results['Actual Return'].rank(ascending=False, method='dense')
predicted_results['Predicted Rank'] = predicted_results['Predicted Return'].rank(ascending=False, method='dense')

# Convert the ranks to integers
predicted_results['Actual Rank'] = predicted_results['Actual Rank'].astype(int)
predicted_results['Predicted Rank'] = predicted_results['Predicted Rank'].astype(int)

In [47]:
predicted_results = predicted_results.sort_values(by='Actual Rank', ascending=True)
predicted_results.head(20)

,Company,Actual Return,Predicted Return,Actual Rank,Predicted Rank
2070,Lendable (London),100.691246,2.742442,1,242
2186,Socure,44.234405,4.294435,2,159
1784,Aviatrix,41.600000,14.448822,3,28
2081,Whoop,38.749741,4.574037,4,147
2093,Weights & Biases,35.880000,7.473577,5,82
2020,Root Insurance (NAS: ROOT),34.627885,4.410215,6,154
2071,Better (NAS: BETR),31.500000,4.578270,7,146
1786,Mux,28.848946,14.845385,8,26
1787,DataRobot,26.828270,21.933786,9,9
1790,Deel,21.043478,6.424968,10,94


In [48]:
predicted_results.to_excel('predicted_results_random_forest_regressor_baseline.xlsx', index=False)

## Saving Feature Importances Assigned by Model

In [51]:
feature_importances = rf.feature_importances_

x_train = X_train.drop('COMPANIES', axis=1)

feature_importance_df = pd.DataFrame({
    'Feature': x_train.columns,  # Assuming X_train is your training data
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [54]:
feature_importance_df

,Feature,Importance
4,RAISED_TO_DATE,0.248435
0,DAYS_FROM_DEAL,0.214312
1,DEAL_SIZE,0.133578
3,POST_VALUATION,0.112391
8,Market Size Estimate,0.087376
5,FOUNDED_YEAR,0.047485
2,PRE_MONEY_VALUATION,0.040379
6,DEAL_MONTH,0.031552
7,PRIMARY_INDUSTRY_SECTOR,0.022342
13,Firms: Best Investors,0.010839


In [50]:
feature_importance_df.to_excel('feature_importance_df_random_forest_regressor_baseline.xlsx', index=False)